In [10]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# #class representing an infected person
# class Infected:
#     def __init__(self, beta, gamma, t):
#         self.beta = beta
#         self.gamma = gamma
#         self.t = t #time infected

In [70]:
###################################################
# original implementation. 
# done purely by "generation order" instead of "time order".        
###################################################

def sim_gen(beta_0, gamma_0, mu_1=0, mu_2=0):
    
    outbreak_thresh = 500 #call it an outbreak if this many people are infected
    
    #store active cases as an array of triples
    #each triple looks like [beta, gamma, time of infection] and represents one infected person
    infecteds=[[beta_0, gamma_0, 0]]
    
    N_infected = len(infecteds) #keep track of number of infected people
    
    t_ext = None #time of extinction, will fill this value and return it if the infection goes extinct
    
    while True: 
        
        #print("\n", "infecteds now = ", infecteds)
        
        #keeps track of indices of recovered people
        #will delete them from array of infecteds at the end of each while loop iteration
        recovereds = []
        
        for i, person in enumerate(infecteds): #go through and branch each person in array of infecteds

            #print("\n", "branching person", i)
            
            [beta, gamma, t] = person
            
            while True:
                                
                #interevent time
                #note this is interevent time for ONE person, NOT for entire population
                dt = np.random.exponential(scale=1/(beta + gamma))
                new_t = t + dt #time of new event
                
                #pick which new event happens, transmission or recovery         
                ev = np.random.rand() #draw number from uniform distribution over [0, 1)
                prob_trans = beta / (beta+gamma)

                if (ev < prob_trans): #transmission
                    
                    #print("person", i, " is transmitting")
                    
                    #pick beta and gamma for new case
                    #mutation is a number drawn from normal distribution with std dev mu_1 or mu_2
                    #don't allow negative beta.
                    #don't allow gamma to be less than a small value, the natural death rate.
                    mut1 = np.random.normal(loc=0.0, scale=mu_1)
                    new_beta = max(0, beta + mut1)
                    mut2 = np.random.normal(loc=0.0, scale=mu_2)              
                    nat_death = 0.00002366575 #taken from CDC: https://www.cdc.gov/nchs/fastats/deaths.htm & scaled to be daily rate instead of yearly
                    new_gamma = max(nat_death, gamma + mut2)
                    
                    #append new person to array of infecteds
                    infecteds = np.append(infecteds, [[new_beta, new_gamma, new_t]], axis=0)
                    N_infected += 1

                    #print("new infection: ", [new_beta, new_gamma, new_t])

                else: #recovery
                    
                    #print("person", i, " recovers") 
                    
                    N_infected -= 1
                    
                    if N_infected == 0:
                        t_ext = new_t #time of extinction
                    
                    #mark index of this recovered person, in order to delete them later
                    #(we don't delete yet, because that will mess up the indexing of the for loop)
                    recovereds.append(i)
                    break #skip to next person

        #after each time we complete a round of branching everybody:
        #1. update the array of infecteds by deleting recoveries
        infecteds = np.delete(infecteds, recovereds, axis=0)
        #2. check if we have extinction/outbreak yet
        if N_infected == 0:
            #print("\n", "extinction")
            return [0, t_ext]
        elif N_infected >= outbreak_thresh:
            #print("\n", "outbreak!")
            return [1]

In [71]:
###############################################################
# second implementation. 
# done by "approximate time order".
# i.e. fully branch one person until recovery, sort list of infecteds by time infected, and repeat           #
###############################################################

def sim_approx_time(beta_0, gamma_0, mu_1=0, mu_2=0):
    
    outbreak_thresh = 500 #call it an outbreak if this many people are infected
    
    #store active cases as an array of triples
    #each triple looks like [beta, gamma, time of infection] and represents one infected person
    infecteds=np.array([[beta_0, gamma_0, 0]])
    N_infected = 1 #keep track of number of infected people. 
    
    t_ext = None #time of extinction, will fill this value and return it if the infection goes extinct
    
    while True: 
        
        #print("\n", "infecteds now = ", infecteds)
        
        person = infecteds[0] #branch person 0. note infecteds is maintained in order of increasing time of infection. 
        [beta, gamma, t] = person
        #print("\n", "branching person who was infected at time", t)

        while True: #branch person 0 until they recover.

            #interevent time
            #note this is interevent time for ONE person, NOT for entire population
            dt = np.random.exponential(scale=1/(beta + gamma))
            new_t = t + dt

            #pick which event happens, transmission or recovery         
            ev = np.random.rand() #draw number from uniform distribution over [0, 1)
            prob_trans = beta / (beta+gamma) #probability of transmission
            
            if (ev < prob_trans): #transmission

                #print("transmitting")

                #pick beta and gamma for new case
                #mutation is a number drawn from normal distribution with std dev mu_1 or mu_2
                #don't allow negative beta.
                #don't allow gamma to be less than a small value, the natural death rate.
                mut1 = np.random.normal(loc=0.0, scale=mu_1)
                new_beta = max(0, beta + mut1)
                mut2 = np.random.normal(loc=0.0, scale=mu_2)              
                nat_death = 0.00002366575 #taken from CDC: https://www.cdc.gov/nchs/fastats/deaths.htm & scaled to be daily rate instead of yearly
                new_gamma = max(nat_death, gamma + mut2)

                N_infected += 1
                
                #print(N_infected)

                #check if we have an outbreak
                if N_infected >= outbreak_thresh:
                    
                    #print("\n", "outbreak!")
                    
                    return [1]

                #append new person to array of infecteds
                infecteds = np.append(infecteds, [[new_beta, new_gamma, new_t]], axis=0)
                
                #print("new infection: ", [new_beta, new_gamma, new_t])

            else: #recovery

                #print("recovering") 

                N_infected -= 1
                
                #print(N_infected)

                #check if we have extinction       
                if N_infected == 0:
                    t_ext = new_t #time of extinction
                    
                    #print("\n", "extinction")
                    
                    return [0, t_ext]

                #delete the person 
                infecteds = np.delete(infecteds, 0, axis=0)
                
                break #skip to next person

            #sort the array of infecteds in increasing order of time infected. 
            sorted_indices = np.argsort(infecteds[:,2])
            infecteds = infecteds[sorted_indices]        

In [72]:
###################################################
# third implementation.
# done by exact time order.
###################################################

def sim_time(beta_0, gamma_0, mu_1=0, mu_2=0):
    
    outbreak_thresh = 500 #call it an outbreak if this many people are infected
    
    t = 0 #initialize variable to keep track of time
        
    #initialize matrix of active cases. each case is a row of length 3.
    # each row looks like [beta, gamma, time of infection] of that person. 
    infecteds=np.array([[beta_0, gamma_0, t]])
    
    #initialize variables to keep track of sums of beta and gamma over all currently infected people    
    beta_sum = beta_0
    gamma_sum = gamma_0
    
    #initial variable to count total number of infected
    N = 1
    
    while True:
        
        #grab array of all betas and array of all gammas
        betas = infecteds[:,0]
        gammas = infecteds[:,1]
        
        #compute interevent time
        
        #rate is sum of beta and gamma over all infected people
        overall_rate = beta_sum + gamma_sum
        #draw from exponential distribution with this rate
        dt = np.random.exponential(scale=1/overall_rate)
        t += dt
        
        #figure out who the event happened to
        i = np.random.choice(N, p=(betas+gammas)/overall_rate) #index of that person
        
        #grab their specific beta and gamma
        beta = betas[i]
        gamma = gammas[i]
        
        #figure out what they did, transmit or recover. 
        ev = np.random.rand() #draw from uniform distribution over [0, 1)
        prob_trans = beta / (beta + gamma) #probability of transmission

        if (ev < prob_trans): #transmission

            #pick beta and gamma for new case
            #mutation is a number drawn from normal distribution with std dev mu_1 or mu_2
            #don't allow negative beta.
            #don't allow gamma to be less than a small value, the natural death rate.
            mut1 = np.random.normal(loc=0.0, scale=mu_1)
            new_beta = max(0, beta + mut1)
            mut2 = np.random.normal(loc=0.0, scale=mu_2)              
            nat_death = 0.00002366575 #taken from CDC: https://www.cdc.gov/nchs/fastats/deaths.htm & scaled to be daily rate instead of yearly
            new_gamma = max(nat_death, gamma + mut2)

            #append new case to infecteds array
            infecteds = np.append(infecteds, [[new_beta, new_gamma, t]], axis=0)
            
            #update the sum of beta and gamma
            beta_sum += new_beta
            gamma_sum += new_gamma
            
            #update N
            N += 1

            #check if we have an outbreak
            if N >= outbreak_thresh:

                #print("\n", "outbreak!")

                return [1]

        else: #recovery
            
            #delete them from infecteds array
            infecteds = np.delete(infecteds, i, axis=0)
            
            #update the sum of beta and gamma
            beta_sum -= betas[i]
            gamma_sum -= gammas[i]
            
            #update N
            N -= 1

            #check if we have extinction       
            if N == 0:

                #print("\n", "extinction")

                return [0, t]

The remaining cells are for testing execution time

In [73]:
#parameters for time tests
beta_0 = 0.09
gamma_0 = 0.1
mu_1 = 0.01
mu_2 = 0.01

In [74]:
%%time
sim_gen(beta_0, gamma_0, mu_1, mu_2)

CPU times: user 919 µs, sys: 187 µs, total: 1.11 ms
Wall time: 951 µs


[0, 66.2441020649024]

In [75]:
%%time
sim_approx_time(beta_0, gamma_0, mu_1, mu_2)

CPU times: user 308 µs, sys: 93 µs, total: 401 µs
Wall time: 338 µs


[0, 25.876468889792797]

In [76]:
%%time
sim_time(beta_0, gamma_0, mu_1, mu_2)

CPU times: user 267 µs, sys: 42 µs, total: 309 µs
Wall time: 282 µs


[0, 2.2500919817342324]

In [33]:
#####################################################
# runs many simulations WITHOUT recurrent spillover #
#####################################################

def sim_percentage_gen(beta_0, gamma_0, mu_1=0, mu_2=0, N_sims=1000):   
    N_outbreaks = 0  
    for i in range(N_sims):
        if sim_gen(beta_0, gamma_0, mu_1, mu_2) == [1]: N_outbreaks += 1
    return N_outbreaks/N_sims

def sim_percentage_approx_time(beta_0, gamma_0, mu_1=0, mu_2=0, N_sims=1000):   
    N_outbreaks = 0  
    for i in range(N_sims):
        if sim_approx_time(beta_0, gamma_0, mu_1, mu_2) == [1]: N_outbreaks += 1
    return N_outbreaks/N_sims

def sim_percentage_time(beta_0, gamma_0, mu_1=0, mu_2=0, N_sims=1000):   
    N_outbreaks = 0  
    for i in range(N_sims):
        if sim_time(beta_0, gamma_0, mu_1, mu_2) == [1]: N_outbreaks += 1
    return N_outbreaks/N_sims

In [ ]:
%%time
sim_percentage_gen(beta_0, gamma_0, mu_1, mu_2, N_sims=1000)

In [ ]:
%%time
sim_percentage_approx_time(beta_0, gamma_0, mu_1, mu_2, N_sims=1000)

In [ ]:
%%time
sim_percentage_time(beta_0, gamma_0, mu_1, mu_2, N_sims=1000)